In [1]:
import os
import sys
import pandas as pd
from xbbg import blp
sys.path.append('../src')
import utils

etf_ticker = 'EFA'
asof = '2025-09-25'

hold_df = pd.read_csv(f'cached_holdings/{etf_ticker}_{asof}.csv')

In [3]:
import numpy as np

hold_df['otc'] = np.nan
hold_df.loc[hold_df['adr_exchange']=='OOTC', 'otc'] = 'yes'
hold_df.loc[hold_df['adr_exchange']!='OOTC', 'otc'] = 'no'
hold_df.groupby('otc')['adr_turnover'].sum()

/tmp/ipykernel_3722940/1342923741.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hold_df.loc[hold_df['adr_exchange']=='OOTC', 'otc'] = 'yes'


otc
no     7.777276e+09
yes    1.346104e+09
Name: adr_turnover, dtype: float64

In [ ]:
filtered = hold_df[hold_df['adr'].notnull() & (hold_df['adr_exchange'] != 'OOTC') & (hold_df['adr_turnover'] > 5e6)].copy()

In [11]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG','AUSTRALIA']),'index_future_bbg'] = 'Z '
filtered.loc[filtered['country'].isin(['JAPAN']),'index_future_bbg'] = 'NH'
filtered.loc[filtered['index_future_bbg'].isna(),'index_future_bbg'] = 'VG'

In [12]:
filtered.loc[filtered['exchange'] == 'MTAA', 'exchange'] = 'XMIL'

In [13]:
filtered.loc[filtered['country'].isin(['BRITAIN','HONG KONG']),'market_etf_hedge'] = 'EWU'
filtered.loc[filtered['country'].isin(['JAPAN']),'market_etf_hedge'] = 'EWJ'
filtered.loc[filtered['country'].isin(['AUSTRALIA']),'market_etf_hedge'] = 'EWA'
filtered.loc[filtered['market_etf_hedge'].isna(),'market_etf_hedge'] = 'FEZ'

In [14]:
filtered[filtered['country']=='AUSTRALIA'][['id','adr','turnover_usd','adr_turnover']]

,id,adr,turnover_usd,adr_turnover
274,BHP AU Equity,BHP US Equity,2.539230e+08,1.667997e+08
682,WDS AU Equity,WDS US Equity,8.251675e+07,1.136334e+07


In [16]:
filtered[filtered['currency'].isin(['AUD','JPY'])]['adr_turnover'].sum()

543300274.1384615

In [17]:
filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])].to_csv('../data/raw/all_adr_filtered.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','JPY'])].to_csv('../data/raw/adr_top_three_currencies.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR'])].to_csv('../data/raw/adr_gbp_eur.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])].to_csv('../data/raw/all_europe.csv', index=False)
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK','AUD','JPY'])].to_csv('../data/raw/all_europe_asia.csv', index=False)

'Z '

In [8]:
filtered[filtered['currency'].isin(['GBp','EUR'])]['adr_turnover'].sum()/filtered[filtered['currency'].isin(['GBp','EUR','JPY','CHF','AUD','DKK'])]['adr_turnover'].sum()

0.7655713455345836

In [ ]:
filtered[filtered['country']=='D']['adr_turnover'].sum()

178163000.0153846

In [27]:
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])]['exchange'].value_counts()

exchange
XLON    22
XAMS     4
XETR     3
XBRU     2
XMAD     2
XMIL     2
XCSE     2
XSWX     2
XPAR     2
XOSL     1
XSTO     1
XHEL     1
XDUB     1
Name: count, dtype: int64

In [29]:
filtered[filtered['currency'].isin(['GBp','EUR','SEK','NOK','CHF','DKK'])]['exchange'].value_counts()

exchange
XLON    22
XAMS     4
XETR     3
XBRU     2
XMAD     2
XMIL     2
XCSE     2
XSWX     2
XPAR     2
XOSL     1
XSTO     1
XHEL     1
XDUB     1
Name: count, dtype: int64

In [39]:
import pandas_market_calendars as mcal

mcal.get_calendar('XCSE').close_time

datetime.time(17, 0, tzinfo=zoneinfo.ZoneInfo(key='Europe/Copenhagen'))

In [43]:
for t in filtered[filtered['currency'].isin(['SEK','NOK','CHF','DKK'])]['adr'].str.replace(' US Equity','').tolist():
    print(f'- {t}')

- EQNR
- ERIC
- GMAB
- NVS
- NVO
- UBS


In [14]:
otc = filtered = hold_df[hold_df['adr'].notnull() & (hold_df['adr_exchange'] == 'OOTC') & (hold_df['adr_turnover'] > 1e6)]


In [24]:
from linux_xbbg import blp

isin = blp.bds(otc['adr'].tolist(), 'ID_ISIN')

In [27]:
otc = otc.merge(isin.rename(columns={'value':'isin'}), left_on='adr', right_index=True, how='left')

In [32]:
otc.to_clipboard(index=False)

In [ ]:

otc[['adr','isin']].sort_values('adr').to_clipboard(index=False)